This notebook is used to query the ambient temperature based on the GPS locations

Right now, it does not work.

In [3]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 52.52,
	"longitude": 13.41,
	"hourly": "temperature_2m",
	"start_date": "2023-01-05",
	"end_date": "2023-09-06"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°E {response.Longitude()}°N")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s"),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s"),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

Coordinates 52.52000045776367°E 13.419998168945312°N
Elevation 38.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
                    date  temperature_2m
0    2023-01-05 00:00:00             NaN
1    2023-01-05 01:00:00             NaN
2    2023-01-05 02:00:00             NaN
3    2023-01-05 03:00:00             NaN
4    2023-01-05 04:00:00             NaN
...                  ...             ...
5875 2023-09-06 19:00:00       22.245501
5876 2023-09-06 20:00:00       20.945499
5877 2023-09-06 21:00:00       19.145500
5878 2023-09-06 22:00:00       18.145500
5879 2023-09-06 23:00:00       17.395500

[5880 rows x 2 columns]


- Le query marche pour une longitude, et latitude donnee

- A tester: faire une query pour chaque timestamp, lon, lat

# Make it a function to loop over all samples

In [ ]:
# TODO

# Does not work

In [ ]:
import requests
import datetime

def get_historical_weather(api_key, location, timestamp):
    # Calculate the timestamp for 2 months ago
    two_months_ago = datetime.datetime.fromtimestamp(timestamp) - datetime.timedelta(days=60)
    timestamp_two_months_ago = int(two_months_ago.timestamp())

    # Define the base URL for the OpenWeatherMap historical data API
    base_url = "https://api.openweathermap.org/data/3.0/onecall/timemachine?"

    # Prepare the parameters for the API request
    params = {
        "lat": location["lat"],
        "lon": location["lon"],
        "dt": timestamp_two_months_ago,
        "appid": api_key,
        "units": "metric",  # You can change this to "imperial" for Fahrenheit
    }

    try:
        # Send a GET request to the OpenWeatherMap historical data API
        response = requests.get(base_url, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()

            # Extract the temperature from the historical data
            temperature = data["hourly"][0]["temp"]

            return {
                "temperature": temperature
            }
        else:
            return {
                "error": "Failed to fetch historical weather data. Check your location or API key."
            }

    except Exception as e:
        return {
            "error": str(e)
        }

# Example usage:
if __name__ == "__main__":
    api_key = "028f7fc14dd4849d8a88fcb50968a6b3"
    location = { x
        "lat": 45,  # Replace with the latitude of your location
        "lon": 54,  # Replace with the longitude of your location
    }
    timestamp = 1629986400  # Replace with the timestamp you want to query (2 months ago)

    weather_data = get_historical_weather(api_key, location, timestamp)

    if "error" in weather_data:
        print("Error:", weather_data["error"])
    else:
        temperature = weather_data["temperature"]
        print(f"Temperature at {location} 2 months ago: {temperature}°C")


In [ ]:
import requests

def get_current_weather(api_key, location):
    # Define the base URL for the OpenWeatherMap API
    base_url = "http://api.openweathermap.org/data/2.5/weather?"

    # Prepare the parameters for the API request
    params = {
        "q": location,
        "appid": api_key,
        "units": "metric",  # You can change this to "imperial" for Fahrenheit
    }

    try:
        # Send a GET request to the OpenWeatherMap API
        response = requests.get(base_url, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()

            # Extract relevant weather information
            temperature = data["main"]["temp"]
            weather_description = data["weather"][0]["description"]

            return {
                "temperature": temperature,
                "weather_description": weather_description,
            }
        else:
            return {
                "error": "Failed to fetch weather data. Check your location or API key."
            }

    except Exception as e:
        return {
            "error": str(e)
        }

# Example usage:
if __name__ == "__main__":
    api_key = "028f7fc14dd4849d8a88fcb50968a6b3"
    location = "Brussels"  # Replace with the location you want to check

    weather_data = get_current_weather(api_key, location)

    if "error" in weather_data:
        print("Error:", weather_data["error"])
    else:
        temperature = weather_data["temperature"]
        description = weather_data["weather_description"]
        print(f"Current temperature in {location}: {temperature}°C")
        print(f"Weather: {description}")


In [ ]:
import requests
import datetime

def get_historical_weather(api_key, location, timestamp):
    # Calculate the timestamp for 2 months ago
    two_months_ago = datetime.datetime.fromtimestamp(timestamp) - datetime.timedelta(days=60)
    two_months_ago_end = two_months_ago + datetime.timedelta(days=1)
    timestamp_two_months_ago = int(two_months_ago.timestamp())
    timestamp_two_months_ago_end = int(two_months_ago_end.timestamp())

    # Define the base URL for the OpenWeatherMap historical data API
    base_url = 'https://history.openweathermap.org/data/2.5/history/city?'
    #base_url = "https://api.openweathermap.org/data/3.0/onecall/timemachine?"

    # Prepare the parameters for the API request
    params = {
        "lat": location["lat"],
        "lon": location["lon"],
        "start": timestamp_two_months_ago,
        "end": timestamp_two_months_ago_end,
        "appid": api_key,
        'type':'hour',
        "units": "metric",  # You can change this to "imperial" for Fahrenheit
    }

    try:
        # Send a GET request to the OpenWeatherMap historical data API
        response = requests.get(base_url, params=params)
        print(response)
        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()

            # Extract the temperature from the historical data
            temperature = data["hourly"][0]["temp"]

            return {
                "temperature": temperature
            }
        else:
            return {
                "error": "Failed to fetch historical weather data. Check your location or API key."
            }

    except Exception as e:
        return {
            "error": str(e)
        }

# Example usage:
if __name__ == "__main__":
    api_key = "028f7fc14dd4849d8a88fcb50968a6b3"
    location = {
        "lat": 45,  # Replace with the latitude of your location
        "lon": 54,  # Replace with the longitude of your location
    }
    timestamp = 1629986400  # Replace with the timestamp you want to query (2 months ago)

    weather_data = get_historical_weather(api_key, location, timestamp)

    if "error" in weather_data:
        print("Error:", weather_data["error"])
    else:
        temperature = weather_data["temperature"]
        print(f"Temperature at {location} 2 months ago: {temperature}°C")


In [ ]:
import requests
import datetime

def get_historical_weather(api_key, location, timestamp):
    # Calculate the timestamp for 2 months ago
    two_months_ago = datetime.datetime.fromtimestamp(timestamp) - datetime.timedelta(days=60)
    two_months_ago_end = two_months_ago + datetime.timedelta(days=1)
    timestamp_two_months_ago = int(two_months_ago.timestamp())
    timestamp_two_months_ago_end = int(two_months_ago_end.timestamp())

    # Define the base URL for the OpenWeatherMap historical data API
    base_url = "https://api.openweathermap.org/data/3.0/onecall?"

    # Prepare the parameters for the API request
    params = {
        "lat": location["lat"],
        "lon": location["lon"],
        #"start": timestamp_two_months_ago,
        #"end": timestamp_two_months_ago_end,
        "appid": api_key,
        #'type':'hour',
        "units": "metric",  # You can change this to "imperial" for Fahrenheit
    }

    try:
        # Send a GET request to the OpenWeatherMap historical data API
        response = requests.get(base_url, params=params)
        print(response)
        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()

            # Extract the temperature from the historical data
            temperature = data["hourly"][0]["temp"]

            return {
                "temperature": temperature
            }
        else:
            return {
                "error": "Failed to fetch historical weather data. Check your location or API key."
            }

    except Exception as e:
        return {
            "error": str(e)
        }

# Example usage:
if __name__ == "__main__":
    api_key = "028f7fc14dd4849d8a88fcb50968a6b3"
    location = {
        "lat": 45,  # Replace with the latitude of your location
        "lon": 54,  # Replace with the longitude of your location
    }
    timestamp = 1629986400  # Replace with the timestamp you want to query (2 months ago)

    weather_data = get_historical_weather(api_key, location, timestamp)

    if "error" in weather_data:
        print("Error:", weather_data["error"])
    else:
        temperature = weather_data["temperature"]
        print(f"Temperature at {location} 2 months ago: {temperature}°C")
